In [59]:
import torch

from torch import nn
from torch.autograd import Variable

In [60]:
input_parameters = 5
layer_size = 5
layer_count = 3
classification_size = 1
learning_rate = 0.001

In [61]:
graph_stat = Variable(torch.ones(input_parameters, layer_size))

In [62]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.linear0 = torch.nn.Linear(layer_size, layer_size)
        self.linear1 = torch.nn.Linear(layer_size, layer_size)
        self.linear2 = torch.nn.Linear(layer_size, classification_size)

    def forward(self, x):
        x = self.linear0(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [63]:
model = NeuralNet()

result = model(graph_stat)
result

tensor([[0.4515],
        [0.4515],
        [0.4515],
        [0.4515],
        [0.4515]], grad_fn=<AddmmBackward0>)

In [64]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [65]:
import pandas
from pandas import DataFrame


def load_dataset(filename: str) -> tuple[list, list]:
    df = pandas.read_csv(filename)
    df.reset_index()
    return [torch.Tensor(v) for v in df.drop('label', axis=1).to_numpy()[:, 1:]], \
           [torch.Tensor([v]) for v in df['label'].to_numpy()]

def f(i, j) -> list[float]:
    return i * 10 + j * j

# dataset = [torch.Tensor([f(i, j) for j in range(input_parameters)]) + torch.randn(input_parameters) for i in range(samples)]
# dataset_labels = [torch.Tensor([i % 3]) for i in range(samples)]
dataset, dataset_labels = load_dataset('dataset.csv')
print(dataset)
print(dataset_labels)

[tensor([41.,  1., 24.,  1., 27.]), tensor([150.,   1.,  60.,   1.,  42.]), tensor([129.,   1.,  42.,   1.,  48.]), tensor([131.,   1.,  40.,   1.,  52.]), tensor([198.,   1.,  64.,   1.,  40.]), tensor([129.,   1.,  42.,   1.,  58.]), tensor([139.,   1.,  46.,   1.,  64.]), tensor([46.,  1., 27.,  1., 32.]), tensor([141.,   1.,  42.,   1.,  59.]), tensor([135.,   1.,  43.,   1.,  58.]), tensor([131.,   1.,  36.,   1.,  52.]), tensor([40.,  1., 24.,  1., 26.]), tensor([151.,   1.,  45.,   1.,  63.]), tensor([117.,   1.,  34.,   1.,  45.]), tensor([40.,  1., 23.,  1., 26.]), tensor([131.,   1.,  44.,   1.,  57.]), tensor([44.,  1., 26.,  1., 30.]), tensor([115.,   1.,  42.,   1.,  45.]), tensor([143.,   1.,  43.,   1.,  55.]), tensor([38.,  1., 24.,  1., 24.]), tensor([131.,   1.,  41.,   1.,  57.]), tensor([139.,   1.,  41.,   1.,  54.]), tensor([129.,   1.,  43.,   1.,  58.]), tensor([113.,   1.,  37.,   1.,  47.]), tensor([41.,  1., 28.,  1., 27.]), tensor([141.,   1.,  48.,   1.,  6

In [66]:
bound = int(len(dataset) * 0.8)
train = list(zip(dataset[:bound], dataset_labels[:bound]))
test = list(zip(dataset[bound:], dataset_labels[bound:]))

In [67]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
total_step = len(train)
for epoch in range(num_epochs):
    for i, (image, label) in enumerate(train):
        # Move tensors to the configured device
        image = image.to(device)
        label = label.to(device)

        # Forward pass
        outputs = model(image)
        loss = criterion(outputs, label)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/10], Step [100/160], Loss: -0.0000
Epoch [2/10], Step [100/160], Loss: -0.0000
Epoch [3/10], Step [100/160], Loss: -0.0000
Epoch [4/10], Step [100/160], Loss: -0.0000
Epoch [5/10], Step [100/160], Loss: -0.0000
Epoch [6/10], Step [100/160], Loss: -0.0000
Epoch [7/10], Step [100/160], Loss: -0.0000
Epoch [8/10], Step [100/160], Loss: -0.0000
Epoch [9/10], Step [100/160], Loss: -0.0000
Epoch [10/10], Step [100/160], Loss: -0.0000


In [68]:
# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for image, label in test:
        image = image.to(device)
        label = label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 0)
        total += label.size(0)
        correct += (predicted == label).item()

    print(f'Accuracy of the network on the {len(test)} test images: {100 * correct / total} %')

Accuracy of the network on the 40 test images: 85.0 %
